In [8]:
# Import required package
using DifferentialEquations
using COBREXA
using DataFrames
using Tulip
using CSV
using Plots
using Colors
using ModelingToolkit

In [10]:
repression(x, k, theta, n) = k/(1+(x/theta)^n)
reversible_michaelismenten(x, y, vm, keq, kmx, kmy) = (vm*(x - (y/keq)))/(x + kmx*(1+(y/kmy)))
hillequation(x, vm, n, km) = (vm*x)^n / (km^n + x^n)
michaelismenten(x, vm, km) = (vm*x)/(km+x)
michaelismenten_substrateactivation(x, vm, km, a, ka) = ((vm * (1+ (a*x)/(ka + x)))*x)/(km + x)

function native_metabolism_loop(du, u, p, t)
    lam, v_in, v_out, k, theta = p
    g6p, f6p, pgi = u
    kcat_pgi = 42. * 60 * 60#EC 5.3.1.9 in Brenda [1/s]
    keq_pgi = 0.3 #[no units]
    km_pgi_g6p = 0.28 #[mM]
    km_pgi_f6p = 0.147 #[mM]
    n = 2. #[no units]
    
    v_pgi = pgi*reversible_michaelismenten(g6p, f6p, kcat_pgi, keq_pgi, km_pgi_g6p, km_pgi_f6p)
    du[1] = v_in - v_pgi - lam*g6p
    du[2] = v_pgi - lam*f6p - v_out
    du[3] = repression(f6p, k, theta, n) - lam*pgi 
end

function native_metabolism(du, u, p, t)
    lam, v_in, v_out = p
    g6p, f6p = u
    keq_pgi = 0.3 #[no units]
    km_pgi_g6p = 0.28 #[mM]
    km_pgi_f6p = 0.147 #[mM]
    vm_pgi = 1.1094 * 60 * 60#[mM/s]
    
    v_pgi = reversible_michaelismenten(g6p, f6p, vm_pgi, keq_pgi, km_pgi_g6p, km_pgi_f6p)
    du[1] = v_in - v_pgi - lam*g6p
    du[2] = v_pgi - lam*f6p - v_out
end

function plot_results(fba_data, ode_data)
    p1 = plot(fba_data.time, [fba_data.lam], ylabel = "Growth [mM/hr]", lw = 3, legend=false, linecolor="black")
    p2 = plot(fba_data.time, [fba_data.v_in, fba_data.v_pgi, fba_data.v_out], ylabel = "Fluxes [mM/hr]", lw = 3, label=["v_in" "v_pgi" "v_out"], legend=true)
    p3 = plot(ode_data.time, [ode_data.g6p, ode_data.f6p], ylabel = "Metabolites [mM]", lw = 3, label=["g6p" "f6p"], legend=true, linecolor=["goldenrod" "purple"])
    p4 = plot(ode_data.time, [ode_data.pgi], ylabel = "Enzymes [mM]", lw = 3, label="pgi", legend=true, linecolor="red3")
    plot(p1, p2, p3, p4, layout = (2, 2))
end

function glucaric_acid(du, u, p, t)
    lam, v_in, A, W = p
    g, g6p, f6p, mi, ino1, miox = u

    n_ino1, theta_ino1, k_ino1 = W[1]
    n_miox, theta_miox, k_miox = W[2]

    vm_ino1 = 0.2616  * 60 * 60 #[mM/s]
    km_ino1_g6p = 1.18 #[mM]
    vm_t_mi = 0.045 * 60 * 60 #[mM/s]
    km_t_mi = 15 #[mM]
    vm_miox = 0.2201 * 60 * 60 #[mM/s]
    km_miox_mi = 24.7 #[mM]
    a_miox = 5.4222 #[no units]
    ka_miox_mi = 20 #[mM]
    vm_glc = 0.1 * 60 * 60#EC 2.7.1.63 [mM/s]
    km_glc = 0.082 #[mM]

    v_glc = michaelismenten(g, vm_glc, km_glc)
    v_ino1 = ino1 * michaelismenten(g6p, vm_ino1, km_ino1_g6p)
    v_pgi = reversible_michaelismenten(g6p, f6p, vm_pgi, keq_pgi, km_pgi_g6p, km_pgi_f6p)
    v_pfk = hilleqn(f6p, vm_pfk, n_pfk, km_pfk_f6p)
    v_tm = michaelismenten(mi, vm_t_mi, km_t_mi)
    v_miox = miox * michaelismenten_substrateactivation(mi, vm_miox, km_miox_mi, a_miox, ka_miox_mi)

    u_ino1_mi = np.sum(A[1]*[activation(mi, k_ino1, theta_ino1, n_ino1), repression(mi, k_ino1, theta_ino1, n_ino1), k_ino1])
    u_miox_mi = np.sum(A[2]*[activation(mi, k_miox, theta_miox, n_miox), repression(mi, k_miox, theta_miox, n_miox), k_miox])


    du[1] = v_in - lam*g - v_glc
    du[2] = v_glc - v_pgi - v_ino1 - lam*g6p
    du[3] = v_pgi - v_pfk - lam*f6p
    du[4] = v_ino1 - v_miox - v_tm - lam*mi
    du[5] = u_ino1_mi  - lam*ino1
    du[6] = u_miox_mi - lam*miox
end

glucaric_acid (generic function with 1 method)

In [22]:

function timing_study(N)
    deltat = 1/(60*60) #genetic timescale, seconds
    dt = 0.001 #kinetic timescale
    k = 1E-7
    theta = 1E-7

    #Run pre-simulation with initial FBA v_in, v_out lam
    model = load_model("e_coli_core.xml") #load core E.coli model
    #run initial FBA
    fluxes = flux_balance_analysis_dict(model, Tulip.Optimizer)
    lam = fluxes["R_BIOMASS_Ecoli_core_w_GAM"]
    v_in = (fluxes["R_GLCpts"] - fluxes["R_G6PDH2r"] - 0.205*fluxes["R_BIOMASS_Ecoli_core_w_GAM"])
    v_out = (fluxes["R_TKT2"] + fluxes["R_FBP"] + fluxes["R_FRUpts2"] - fluxes["R_PFK"] + fluxes["R_TALA"] - 0.0709*fluxes["R_BIOMASS_Ecoli_core_w_GAM"])
    p = [lam, v_in, v_out] 

    tspan = [0., 150] #[hr]
    savetimes = [150]
    u0 = [0., 0.]
    prob = ODEProblem(native_metabolism, u0, tspan, p)
    sol = solve(prob, Tsit5(), reltol=1e-3, abstol=1e-6, saveat=savetimes)

    #Get PGI initial condition from VM/Kcat_pgi (get Kcat from BRENDA)
    pgi_o = 1.1094/42.
    u0 = sol.u[end]
    u0 = [u0[1], u0[2], pgi_o]

    #instantiate initial times
    starttime = 0.
    endtime = starttime + deltat
    tspan = [starttime, endtime]
    savetimes = [endtime]

    p = [lam, v_in, v_out, k, theta] 

    #FBA-ODE optimization loop
    ode_data = DataFrame()
    fba_data = DataFrame("time" => [0], "v_in" => [v_in], "v_pgi" => [fluxes["R_PGI"]], "v_out" => [v_out], "lam" => [lam])

    total_ode_time = 0.0
    total_fba_time = 0.0

    for i in 1:N
        prob = ODEProblem(native_metabolism_loop, u0, tspan, p)
        #Solve ODE
        total_ode_time += @elapsed begin
            sol = solve(prob, Rosenbrock23(), reltol=1e-3, abstol=1e-6, saveat=savetimes)
        end
        
        #Solve for V_PGI from concentrations
        v_pgi = sol.u[end][3]*reversible_michaelismenten(sol.u[end][1], sol.u[end][2], 0.8751, 0.3, 0.28, 0.147)
        v_pgi = v_pgi
        #Save ODE data
        ode_data = vcat(ode_data, DataFrame("time" => sol.t, "g6p" => sol.u[1][1], "f6p" => sol.u[1][2], "pgi" => sol.u[1][3]))
        
        total_fba_time += @elapsed begin
            restricted_fluxes = flux_balance_analysis_dict(model, Tulip.Optimizer, modifications = [change_constraint("R_PGI", lb = v_pgi, ub = v_pgi)])
        end
        #Update parameter and timing values
        lam = restricted_fluxes["R_BIOMASS_Ecoli_core_w_GAM"]
        v_in = (restricted_fluxes["R_GLCpts"] - restricted_fluxes["R_G6PDH2r"] - 0.205*restricted_fluxes["R_BIOMASS_Ecoli_core_w_GAM"])
        v_out = (restricted_fluxes["R_TKT2"] + restricted_fluxes["R_FBP"] + restricted_fluxes["R_FRUpts2"] - restricted_fluxes["R_PFK"] + restricted_fluxes["R_TALA"] - 0.0709*restricted_fluxes["R_BIOMASS_Ecoli_core_w_GAM"])
        p = [lam, v_in, v_out, k, theta] 

        starttime = endtime
        endtime = starttime + deltat
        tspan = [starttime, endtime]
        savetimes = [endtime]
        u0 = sol.u[end]
        #Save FBA data
        fba_data = vcat(fba_data, DataFrame("time" => [starttime], "v_in" => [p[2]], "v_pgi" => [restricted_fluxes["R_PGI"]], "v_out" => [p[3]], "lam" => [p[1]]))
    end

    println("Number of iterations ", N)
    println("Total FBA time ", total_fba_time)
    println("Total ODE time ", total_ode_time)
    println("Fraction of FBA time ", total_fba_time/(total_ode_time+total_fba_time))
end

# plot_results(fba_data, ode_data)

timing_study (generic function with 1 method)

In [24]:
for N in [10, 100, 1000, 1*60*60, 10*60*60]
    timing_study(N)
end

Number of iterations 10
Total FBA time 0.10692087500000001
Total ODE time 0.0009322489999999999
Fraction of FBA time 0.9913563097161655
Number of iterations 100
Total FBA time 0.7909390079999996
Total ODE time 0.004051998999999999
Fraction of FBA time 0.9949030882559405
Number of iterations 1000
Total FBA time 2.764658873000002
Total ODE time 0.02310071599999997
Fraction of FBA time 0.9917135193109365
Number of iterations 3600
Total FBA time 10.425673537000002
Total ODE time 0.09186654800000016
Fraction of FBA time 0.991265395971153
Number of iterations 36000
Total FBA time 120.00652596599859
Total ODE time 1.2351885750000036
Fraction of FBA time 0.9898121815608082
